In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import awkward as ak

from coffea import processor, hist
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.analysis_tools import Weights, PackedSelection
from klepto.archives import dir_archive

import numpy as np

from processor.forward_jet import *


In [ ]:
# load the basic filesets and choose what samples we actually want to load

from Tools.samples import fileset_2018
from processor.default_accumulators import variations, desired_output, add_processes_to_output


In [ ]:
from Tools.helpers import get_scheduler_address

from dask.distributed import Client, progress

scheduler_address = get_scheduler_address()

c = Client(scheduler_address)

c

In [ ]:
%%time
overwrite = True
small = True
local = False
year = 2018

# load the config and the cache
cfg = loadConfig()

cacheName = 'forward'
if small: cacheName += '_small'
cache = dir_archive(os.path.join(os.path.expandvars(cfg['caches']['base']), cacheName), serialized=True)

fileset = {
    #'tW_scattering': fileset_2018['tW_scattering'],
    'topW_v3': fileset_2018['topW_v3'],
    'ttbar': fileset_2018['ttbar2l'], # dilepton ttbar should be enough for this study.
    'MuonEG': fileset_2018['MuonEG'],
    'DoubleMuon': fileset_2018['DoubleMuon'],
    'EGamma': fileset_2018['EGamma'],
    'WW': fileset_2018['WW'],
    'WZ': fileset_2018['WZ'],
    'DY': fileset_2018['DY'],
}

fileset = make_small(fileset, small)

add_processes_to_output(fileset, desired_output)

for rle in ['run', 'lumi', 'event']:
    desired_output.update({
            'MuonEG_%s'%rle: processor.column_accumulator(np.zeros(shape=(0,))),
            'EGamma_%s'%rle: processor.column_accumulator(np.zeros(shape=(0,))),
            'DoubleMuon_%s'%rle: processor.column_accumulator(np.zeros(shape=(0,))),
            })

histograms = sorted(list(desired_output.keys()))

if local:
    exe_args = {
        'workers': 3,
        'function_args': {'flatten': False},
        "schema": NanoAODSchema,
    }
    exe = processor.futures_executor
    
else:
    exe_args = {
        'client': c,
        'function_args': {'flatten': False},
        "schema": NanoAODSchema,
    }
    exe = processor.dask_executor



if not overwrite:
    cache.load()

#if cfg == cache.get('cfg') and histograms == cache.get('histograms') and cache.get('simple_output'):
if cfg == cache.get('cfg') and cache.get('simple_output'):
    output = cache.get('simple_output')

else:
    print ("I'm running now")
    
    output = processor.run_uproot_job(
        fileset,
        "Events",
        forwardJetAnalyzer(year=year, variations=variations, accumulator=desired_output),
        exe,
        exe_args,
        chunksize=250000,
    )
    
    cache['fileset']        = fileset
    cache['cfg']            = cfg
    cache['histograms']     = histograms
    cache['simple_output']  = output
    cache.dump()

~22s for baseline code


In [ ]:
# Cutflow
from Tools.helpers import getCutFlowTable

#processes = ['tW_scattering', 'topW_v2']
processes = ['DY', 'ttbar', 'WW', 'WZ', 'MuonEG', 'EGamma', 'DoubleMuon']

# let's use the S/B functionality to get data/MC by defining data (MuonEG) as signal
lines = ['entry']
lines +=   ['filter',
            'lepveto',
            'dilep',
            'p_T(lep0)>25',
            'p_T(lep1)>20',
            'trigger',
            'OS',
            'N_jet>3',
            'N_central>2',
            'N_btag>0',
            'MET>30',
            'N_fwd>0',
            ]


df = getCutFlowTable(output, processes=processes, lines=lines, significantFigures=4,
                    # signal='MuonEG'
                    )
df

5.693 v0.2.3 vs 5.693 in v0.2.2 -> good

PU weight looks good, too.


In [ ]:
df = getCutFlowTable(output, processes=processes, lines=lines, significantFigures=4, absolute=False)
df

/hadoop/cms/store/user/dspitzba/ProjectMetis/TTWJetsToLNuEWK_5f_NLO_RunIIAutumn18_NANO_v2/:
  files: 478
  nEvents: 478000
  name: ProjectMetis_TTWJetsToLNuEWK_5f_NLO_RunIIAutumn18_NANO_v2
  path: /hadoop/cms/store/user/dspitzba/ProjectMetis/TTWJetsToLNuEWK_5f_NLO_RunIIAutumn18_NANO_v2/
  split: 207
  sumWeight: 22576.62849550001
  xsec: 0.0478

filter efficiency: 0.482

/hadoop/cms/store/user/dspitzba/tW_scattering/tW_scattering/nanoAOD/:
  files: 56
  nEvents: 54200
  name: tW_scattering_nanoAOD
  path: /hadoop/cms/store/user/dspitzba/tW_scattering/tW_scattering/nanoAOD/
  split: 56
  sumWeight: 2622.728769570001
  xsec: 0.0478
  
filter efficiency: 0.46

**I need to check where these two samples depart from each other**

Most of the selections are slightly less efficient on the new sample, but they agree within 2 sigma.
In the end it is

7.834 +/- 0.674 (old) vs 7.096 +/- 0.216 (new)

in the OS ttbar selection.

The PU distribution looks a bit odd in the old sample, but otherwise things look good.
(PU mixing has been updated in the new sample to include a larger number of neutrino gun files)


In [ ]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)

from plots.helpers import *

# defining some new axes for rebinning.
N_bins = hist.Bin('multiplicity', r'$N$', 10, -0.5, 9.5)
N_bins_red = hist.Bin('multiplicity', r'$N$', 5, -0.5, 4.5)
mass_bins = hist.Bin('mass', r'$M\ (GeV)$', 20, 0, 200)
pt_bins = hist.Bin('pt', r'$p_{T}\ (GeV)$', 30, 0, 300)
pt_bins_coarse = hist.Bin('pt', r'$p_{T}\ (GeV)$', 10, 0, 300)
eta_bins = hist.Bin('eta', r'$\eta $', 25, -5.0, 5.0)

my_labels = {
    'topW_v2': 'top-W scat.',
    'WW': 'WW',
    'WZ': 'WZ',
    'ttbar': r'$t\bar{t}$',
    'DY': 'Drell-Yan',
    'MuonEG': 'Observation',
    'EGamma': 'Observation',
    'DoubleMuon': 'Observation',
}

my_colors = {
    'topW_v2': '#FF595E',
    'WW': '#34623F',
    'WZ': '#525B76',
    'ttbar': '#1982C4',
    'DY': '#6A4C93',
    'MuonEG': '#000000',
    'EGamma': '#000000',
    'DoubleMuon': '#000000',
}

In [ ]:
makePlot(output, 'PV_npvsGood', 'multiplicity',
         data=['MuonEG', 'DoubleMuon', 'EGamma'],
         bins=None,
         log=True, normalize=True, axis_label=r'$N_{good PV}$',
         new_colors=my_colors, new_labels=my_labels,
         order=['topW_v2', 'WW', 'WZ', 'DY', 'ttbar'],
         omit=[],
        )

In [ ]:
makePlot(output, 'N_ele', 'multiplicity',
         data=['MuonEG', 'DoubleMuon', 'EGamma'],
         bins=N_bins_red,
         log=True, normalize=True, axis_label=r'$N_{good PV}$',
         new_colors=my_colors, new_labels=my_labels,
         order=['topW_v2', 'WW', 'WZ', 'DY', 'ttbar'],
         omit=[],
        )

In [ ]:
makePlot(output, 'N_b', 'multiplicity',
         data=['MuonEG', 'DoubleMuon', 'EGamma'],
         bins=N_bins_red,
         log=True, normalize=True, axis_label=r'$N_{b-tag}$',
         new_colors=my_colors, new_labels=my_labels,
         order=['topW_v2', 'WW', 'WZ', 'DY', 'ttbar'],
         omit=[],
        )

In [ ]:
makePlot(output, 'N_fwd', 'multiplicity',
         data=['MuonEG', 'DoubleMuon', 'EGamma'],
         bins=N_bins_red,
         log=True, normalize=True, axis_label=r'$N_{fwd\ jet}$',
         new_colors=my_colors, new_labels=my_labels,
         order=['topW_v2', 'WW', 'WZ', 'DY', 'ttbar'],
         omit=[],
        )

In [ ]:
makePlot(output, 'N_jet', 'multiplicity',
         data=['MuonEG', 'DoubleMuon', 'EGamma'],
         bins=N_bins,
         log=True, normalize=True, axis_label=r'$N_{jet}$',
         new_colors=my_colors, new_labels=my_labels,
         order=['topW_v2', 'WW', 'WZ', 'DY', 'ttbar'],
         omit=[],
        )

In [ ]:
makePlot(output, 'N_jet', 'multiplicity',
         data=['MuonEG', 'DoubleMuon', 'EGamma'],
         bins=N_bins,
         log=True, normalize=True, axis_label=r'$N_{jet}$',
         new_colors=my_colors, new_labels=my_labels,
         #order=['WW', 'WZ', 'DY', 'ttbar'],
         omit=[],
        )

In [ ]:
makePlot(output, 'lead_lep', 'pt',
         data=['MuonEG', 'DoubleMuon', 'EGamma'],
         bins=pt_bins,
         log=True, normalize=True, axis_label=r'$p_{T}$ (lead lep) (GeV)',
         new_colors=my_colors, new_labels=my_labels,
         order=['WW', 'WZ', 'DY', 'ttbar'],
         signals=['topW_v2'],
         omit=[],
        )

In [ ]:
makePlot(output, 'lead_lep', 'eta',
         data=['MuonEG', 'DoubleMuon', 'EGamma'],
         bins=eta_bins,
         log=True, normalize=True, axis_label=r'$p_{T}$ (lead lep) (GeV)',
         new_colors=my_colors, new_labels=my_labels,
         order=['WW', 'WZ', 'DY', 'ttbar'],
         omit=[],
        )

In [ ]:
makePlot(output, 'fwd_jet', 'pt',
         data=['MuonEG', 'DoubleMuon', 'EGamma'],
         bins=pt_bins,
         log=True, normalize=True, axis_label=r'$p_{T}$ (lead lep) (GeV)',
         new_colors=my_colors, new_labels=my_labels,
         order=['WW', 'WZ', 'DY', 'ttbar'],
         omit=[],
        )

In [ ]:
makePlot(output, 'j1', 'pt',
         data=['MuonEG', 'DoubleMuon', 'EGamma'],
         bins=pt_bins,
         log=True, normalize=True, axis_label=r'$p_{T}$ (lead lep) (GeV)',
         new_colors=my_colors, new_labels=my_labels,
         order=['WW', 'WZ', 'DY', 'ttbar'],
         omit=[],
        )

In [ ]:
makePlot(output, 'PV_npvsGood', 'multiplicity',
         data_sel=None, # use None if you don't use observation
         bins=None, log=False, normalize=True, axis_label=r'$N_{good\ PV}$',
        )

In [ ]:
makePlot(output, 'PV_npvsGood', 'multiplicity',
         #data_sel=None,
         bins=None, log=False, normalize=True, axis_label=r'$N_{good\ PV}$',
         #upHists=['pt_jesTotalUp'], downHists=['pt_jesTotalDown']
        )

In [ ]:
makePlot(output, 'N_fwd', 'multiplicity',
         #data_sel=None,
         bins=N_bins_red, log=False, normalize=True, axis_label=r'$N_{fwd\ jet}$',
         upHists=['pt_jesTotalUp'], downHists=['pt_jesTotalDown']
        )

In [ ]:
makePlot(output, 'N_jet', 'multiplicity',
         #data_sel=None,
         bins=N_bins, log=False, normalize=True, axis_label=r'$N_{jet}$',
         upHists=['pt_jesTotalUp'], downHists=['pt_jesTotalDown']
        )

In [ ]:
makePlot(output, 'N_b', 'multiplicity',
         #data_sel=None,
         bins=N_bins_red, log=False, normalize=True, axis_label=r'$N_{b-tag}$',
         upHists=['pt_jesTotalUp'], downHists=['pt_jesTotalDown']
        )

In [ ]:
makePlot(output, 'N_central', 'multiplicity',
         #data_sel=None,
         bins=N_bins, log=False, normalize=True, axis_label=r'$N_{central\ jet}$',
         upHists=['pt_jesTotalUp'], downHists=['pt_jesTotalDown']
        )

In [ ]:
makePlot(output, 'MET', 'pt',
         #data_sel=None,
         bins=pt_bins_coarse, log=False, normalize=True, axis_label=r'$p_{T}^{miss}\ (GeV)$',
         upHists=['pt_jesTotalUp'], downHists=['pt_jesTotalDown'],
         ratio_range = (0.75,1.25)
        )

In [ ]:
makePlot(output, 'fwd_jet', 'pt',
         #data_sel=None,
         bins=pt_bins_coarse, log=False, normalize=True, axis_label=r'$p_{T,\ fwd\ jet}$ (GeV)',
         upHists=['pt_jesTotalUp'], downHists=['pt_jesTotalDown'],
         save='/home/users/dspitzba/public_html/tW_scattering/dump/fwd_pt_syst_v2'
        )

In [ ]:
jet_name = 'fwd_jet'
sample_name = 'ttbar'

histogram = output[jet_name].project('pt', 'dataset').rebin('pt', pt_bins)
ax = hist.plot1d(histogram[sample_name],overlay="dataset", stack=False, overflow='over')
print ("Central:", sum(histogram[sample_name].sum('dataset', overflow='over').values()[()]))

histogram = output[jet_name+'_pt_jesTotalUp'].project('pt', 'dataset').rebin('pt', pt_bins)
ax = hist.plot1d(histogram[sample_name],overlay="dataset", stack=False, overflow='over')
print ("Up:", sum(histogram[sample_name].sum('dataset', overflow='over').values()[()]))

histogram = output[jet_name+'_pt_jesTotalDown'].project('pt', 'dataset').rebin('pt', pt_bins)
ax = hist.plot1d(histogram[sample_name],overlay="dataset", stack=False, overflow='over')
print ("Down:", sum(histogram[sample_name].sum('dataset', overflow='over').values()[()]))



In [ ]:
makePlot(output, 'fwd_jet', 'eta',
         #data_sel=None,
         bins=eta_bins, log=False, normalize=True, axis_label=r'$p_{T,\ fwd\ jet}$',
         upHists=['pt_jesTotalUp'], downHists=['pt_jesTotalDown'],
         #save='/home/users/dspitzba/public_html/tW_scattering/dump/fwd_pt_syst'
        )

In [ ]:
makePlot(output, 'j1', 'pt',
         #data_sel=None,
         bins=pt_bins, log=False, normalize=True, axis_label=r'$p_{T}$ (leading jet) (GeV)',
         upHists=['pt_jesTotalUp'], downHists=['pt_jesTotalDown'],
         #save='/home/users/dspitzba/public_html/tW_scattering/dump/fwd_pt_syst'
        )

In [ ]:
makePlot(output, 'b1', 'pt',
         #data_sel=None,
         bins=pt_bins, log=False, normalize=True, axis_label=r'$p_{T}$ (leading b-tagged jet) (GeV)',
         upHists=['pt_jesTotalUp'], downHists=['pt_jesTotalDown'],
         #save='/home/users/dspitzba/public_html/tW_scattering/dump/fwd_pt_syst'
        )

In [ ]:
makePlot(output, 'lead_lep', 'pt',
         #data_sel=None,
         bins=pt_bins, log=False, normalize=True, axis_label=r'$p_{T}$ (leading lepton) (GeV)',
         ratio_range = (0.5,1.5)
         )

In [ ]:
makePlot(output, 'trail_lep', 'pt',
         #data_sel=None,
         bins=pt_bins, log=False, normalize=True, axis_label=r'$p_{T}$ (trailing lepton) (GeV)',
         ratio_range = (0.5,1.5)
         )

In [ ]:
makePlot(output, 'electron', 'pt',
         #data_sel=None,
         bins=pt_bins, log=False, normalize=True, axis_label=r'$p_{T}$ (electron) (GeV)',
         ratio_range = (0.5,1.5)
         )

In [ ]:
makePlot(output, 'muon', 'pt',
         #data_sel=None,
         bins=pt_bins, log=False, normalize=True, axis_label=r'$p_{T}$ (muon) (GeV)',
         ratio_range = (0.5,1.5)
         )

It seems like either we have a wrong normalization of ttbar, missing data events, or the lepton SFs are huge (and the PU reweighting also contributes much).

With ttH lepton IDs this is the normalization:

Data: 20146.0 MC: 28085.86

Looser SS ID:

Data: 26242.0 MC: 34977.04

There's some missing 0b simulation, potentially W+jets with a fake?

Let's implement the proper lepton SFs and then see, SFs around 0.9 will already bring data/MC to agreement.

# Some development stuff

In [ ]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

# the below command will change to .from_root in coffea v0.7.0
events = NanoEventsFactory.from_root('/hadoop/cms/store/user/dspitzba/nanoAOD/ttw_samples/topW_v0.2.2/tW_scattering_nanoAOD/nanoSkim_1.root', schemaclass=NanoAODSchema).events()

In [ ]:
weight = Weights(len(events))
#weight.weight()

In [ ]:
import uproot4
#fin = uproot4.open('/hadoop/cms/store/user/dspitzba/nanoAOD/ttw_samples/topW_v0.2.2/tW_scattering_nanoAOD/nanoSkim_1.root')
fin = uproot4.open('/home/users/dspitzba/TTW/CMSSW_10_2_9/src/nanoAOD_37_Skim.root')

In [ ]:
#fin['Events'].show() # this shows all the branches

In [ ]:
from coffea.btag_tools import BTagScaleFactor

In [ ]:
btag_sf = BTagScaleFactor(os.path.expandvars("$TWHOME/Tools/data/btag/DeepJet_102XSF_V2.csv"), "medium")

print("SF:", btag_sf.eval("central", events.Jet.hadronFlavour, abs(events.Jet.eta), events.Jet.pt))
print("systematic +:", btag_sf.eval("up", events.Jet.hadronFlavour, abs(events.Jet.eta), events.Jet.pt))

In [ ]:
sf = btag_sf.eval("central", events.Jet.hadronFlavour, abs(events.Jet.eta), events.Jet.pt, )
len(sf)

In [ ]:
        ev = events
        ## Muons
        muon     = Collections(ev, "Muon", "tightTTH").get()
        vetomuon = Collections(ev, "Muon", "vetoTTH").get()
        dimuon   = choose(muon, 2)
        SSmuon   = ak.any((dimuon['0'].charge * dimuon['1'].charge)>0, axis=1)
        OSmuon   = ak.any((dimuon['0'].charge * dimuon['1'].charge)<0, axis=1)
        leading_muon_idx = ak.singletons(ak.argmax(muon.pt, axis=1))
        leading_muon = muon[leading_muon_idx]
        
        ## Electrons
        electron     = Collections(ev, "Electron", "tightTTH").get()
        vetoelectron = Collections(ev, "Electron", "vetoTTH").get()
        dielectron   = choose(electron, 2)
        SSelectron   = ak.any((dielectron['0'].charge * dielectron['1'].charge)>0, axis=1)
        OSelectron   = ak.any((dielectron['0'].charge * dielectron['1'].charge)<0, axis=1)
        leading_electron_idx = ak.singletons(ak.argmax(electron.pt, axis=1))
        leading_electron = electron[leading_electron_idx]
        
        ## Merge electrons and muons - this should work better now in ak1
        lepton   = ak.concatenate([muon, electron], axis=1)
        dilepton = cross(muon, electron)
        SSlepton = ak.any((dilepton['0'].charge * dilepton['1'].charge)>0, axis=1)
        OSlepton = ak.any((dilepton['0'].charge * dilepton['1'].charge)<0, axis=1)
        leading_lepton_idx = ak.singletons(ak.argmax(lepton.pt, axis=1))
        leading_lepton = lepton[leading_lepton_idx]
        trailing_lepton_idx = ak.singletons(ak.argmin(lepton.pt, axis=1))
        trailing_lepton = lepton[trailing_lepton_idx]

In [ ]:
selection = ((ak.num(electron) + ak.num(muon))==2)

ak.to_numpy(ak.flatten(leading_lepton[selection].pt))

In [ ]:
electron = Collections(events, "Electron", "tight", verbose=True).get()
#muon = Collections(events, "Muon", "tight").get()

In [ ]:
def getPtEtaPhi(coll, pt_var='pt', eta_var='eta', phi_var='phi'):
    #pt = 
    return ak.zip({'pt': getattr(coll, pt_var), 'eta': getattr(coll, eta_var), 'phi': getattr(coll, phi_var)})

In [ ]:
jet = getJets(events)
jet.pt